In [3]:
import sys
sys.path.append('pytorch')

from pytorch.mem_transformer import *
from pytorch import data_utils

In [3]:
class args:
    cuda = False
    n_layer = 2
    n_rel_layer = None #Nothing
    n_head = 4
    d_head = 200      # dim of Query
    d_model = 202    # = d_proj - projection
    d_embed = 198    # dim of embedding b4 attn?
    d_inner = 197    # hidden dim of FF
    dropout = 0
    seed = 42

    n_token = 10000 # dict



device = torch.device("cuda" if args.cuda else "cpu")

B = 4
tgt_len, mem_len, ext_len = 36, 36, 0
data_len = tgt_len * 20



In [4]:
data = torch.LongTensor(data_len*B).random_(0, args.n_token).to(device)
diter = data_utils.LMOrderedIterator(data, B, tgt_len, device=device, ext_len=ext_len)

In [5]:
data.reshape(tgt_len, B, -1).shape

torch.Size([36, 4, 20])

In [6]:
# class MemTransformerLM_mem(nn.Module):
#     def __init__(self, n_token, n_layer, n_head, d_model, d_head, d_inner,
#                  dropout, dropatt, tie_weight=True, d_embed=None, 
#                  div_val=1, tie_projs=[False], pre_lnorm=False,
#                  tgt_len=None, ext_len=None, mem_len=None, num_mem_tokens=None,
#                  cutoffs=[], adapt_inp=False,
#                  same_length=False, attn_type=0, clamp_len=-1, 
#                  sample_softmax=-1):
#         super(MemTransformerLM_mem, self).__init__()
#         self.n_token = n_token

#         d_embed = d_model if d_embed is None else d_embed
#         self.d_embed = d_embed
#         self.d_model = d_model
#         self.n_head = n_head
#         self.d_head = d_head

#         self.word_emb = AdaptiveEmbedding(n_token, d_embed, d_model, cutoffs, 
#                                           div_val=div_val)

#         self.drop = nn.Dropout(dropout)

#         self.n_layer = n_layer

#         self.tgt_len = tgt_len
#         self.mem_len = mem_len
#         self.ext_len = ext_len
#         self.max_klen = tgt_len + ext_len + mem_len
#         self.num_mem_tokens = num_mem_tokens

#         self.attn_type = attn_type

#         self.layers = nn.ModuleList()
#         if attn_type == 0: # the default attention
#             for i in range(n_layer):
#                 self.layers.append(
#                     RelPartialLearnableDecoderLayer(
#                         n_head, d_model, d_head, d_inner, dropout,
#                         tgt_len=tgt_len, ext_len=ext_len, mem_len=mem_len,
#                         dropatt=dropatt, pre_lnorm=pre_lnorm)
#                 )
#         elif attn_type == 1: # learnable embeddings
#             for i in range(n_layer):
#                 self.layers.append(
#                     RelLearnableDecoderLayer(
#                         n_head, d_model, d_head, d_inner, dropout,
#                         tgt_len=tgt_len, ext_len=ext_len, mem_len=mem_len,
#                         dropatt=dropatt, pre_lnorm=pre_lnorm)
#                 )
#         elif attn_type in [2, 3]: # absolute embeddings
#             for i in range(n_layer):
#                 self.layers.append(
#                     DecoderLayer(
#                         n_head, d_model, d_head, d_inner, dropout,
#                         dropatt=dropatt, pre_lnorm=pre_lnorm)
#                 )

#         self.sample_softmax = sample_softmax
#         # use sampled softmax
#         if sample_softmax > 0:
#             self.out_layer = nn.Linear(d_model, n_token)
#             if tie_weight:
#                 self.out_layer.weight = self.word_emb.weight
#             self.tie_weight = tie_weight
#             self.sampler = LogUniformSampler(n_token, sample_softmax)

#         # use adaptive softmax (including standard softmax)
#         else:
#             self.crit = ProjectedAdaptiveLogSoftmax(n_token, d_embed, d_model, 
#                                                     cutoffs, div_val=div_val)

#             if tie_weight:
#                 for i in range(len(self.crit.out_layers)):
#                     self.crit.out_layers[i].weight = self.word_emb.emb_layers[i].weight

#             if tie_projs:
#                 for i, tie_proj in enumerate(tie_projs):
#                     if tie_proj and div_val == 1 and d_model != d_embed:
#                         self.crit.out_projs[i] = self.word_emb.emb_projs[0]
#                     elif tie_proj and div_val != 1:
#                         self.crit.out_projs[i] = self.word_emb.emb_projs[i]

#         self.same_length = same_length
#         self.clamp_len = clamp_len

#         self._create_params()
#         self.init_mem_tokens()

#     def backward_compatible(self):
#         self.sample_softmax = -1

#     def _create_params(self):
#         if self.attn_type == 0: # default attention
#             self.pos_emb = PositionalEmbedding(self.d_model)
#             self.r_w_bias = nn.Parameter(torch.Tensor(self.n_head, self.d_head))
#             self.r_r_bias = nn.Parameter(torch.Tensor(self.n_head, self.d_head))
#         elif self.attn_type == 1: # learnable
#             self.r_emb = nn.Parameter(torch.Tensor(
#                     self.n_layer, self.max_klen, self.n_head, self.d_head))
#             self.r_w_bias = nn.Parameter(torch.Tensor(
#                     self.n_layer, self.n_head, self.d_head))
#             self.r_bias = nn.Parameter(torch.Tensor(
#                     self.n_layer, self.max_klen, self.n_head))
#         elif self.attn_type == 2: # absolute standard
#             self.pos_emb = PositionalEmbedding(self.d_model)
#         elif self.attn_type == 3: # absolute deeper SA
#             self.r_emb = nn.Parameter(torch.Tensor(
#                     self.n_layer, self.max_klen, self.n_head, self.d_head))

#     def reset_length(self, tgt_len, ext_len, mem_len):
#         self.tgt_len = tgt_len
#         self.mem_len = mem_len
#         self.ext_len = ext_len

#     def init_mems(self):
#         if self.mem_len > 0:
#             mems = []
#             param = next(self.parameters())
#             for i in range(self.n_layer+1):
#                 empty = torch.empty(0, dtype=param.dtype, device=param.device)
#                 mems.append(empty)

#             return mems
#         else:
#             return None

#     def init_mem_tokens(self):
#         if self.num_mem_tokens in (None, 0):
#             self.mem_tokens = None
#         else:
#             mem_tokens = [torch.randn(1, self.d_model)] * self.num_mem_tokens
#             # self.mem_tokens = nn.Parameter(torch.cat(mem_tokens, dim=0))
#             self.mem_tokens = torch.cat(mem_tokens, dim=0)

#     def _update_mems(self, hids, mems, qlen, mlen):
#         # does not deal with None
#         if mems is None: return None

#         # mems is not None
#         assert len(hids) == len(mems), 'len(hids) != len(mems)'

#         # There are `mlen + qlen` steps that can be cached into mems
#         # For the next step, the last `ext_len` of the `qlen` tokens
#         # will be used as the extended context. Hence, we only cache
#         # the tokens from `mlen + qlen - self.ext_len - self.mem_len`
#         # to `mlen + qlen - self.ext_len`.
#         with torch.no_grad():
#             new_mems = []
#             end_idx = mlen + max(0, qlen - 0 - self.ext_len)
#             beg_idx = max(0, end_idx - self.mem_len)
#             for i in range(len(hids)):

#                 cat = torch.cat([mems[i], hids[i]], dim=0)
#                 new_mems.append(cat[beg_idx:end_idx].detach())

#         return new_mems

#     def _forward(self, dec_inp, mems=None):

#         word_emb = self.word_emb(dec_inp)
#         print( word_emb.isnan().sum())

#         mlen = mems[0].size(0) if mems is not None else 0
        
#         # Concat with mem_tokens
#         if self.num_mem_tokens not in (0, None):
#             mem_tokens = self.mem_tokens.expand(1, *self.mem_tokens.shape).clone()
#             memory = torch.cat([mem_tokens] * word_emb.shape[0], dim=0)
#             word_emb = torch.cat((memory, word_emb), dim=1)

#         qlen, bsz = dec_inp.size()
#         klen = mlen + qlen
#         if self.same_length:
#             all_ones = word_emb.new_ones(qlen, klen)
#             mask_len = klen - self.mem_len
#             if mask_len > 0:
#                 mask_shift_len = qlen - mask_len
#             else:
#                 mask_shift_len = qlen
#             dec_attn_mask = (torch.triu(all_ones, 1+mlen)
#                     + torch.tril(all_ones, -mask_shift_len)).byte()[:, :, None] # -1
#         else:
#             dec_attn_mask = torch.triu(
#                 word_emb.new_ones(qlen, klen), diagonal=1+mlen).byte()[:,:,None]
        
#         hids = []
#         if self.attn_type == 0: # default
#             pos_seq = torch.arange(klen-1, -1, -1.0, device=word_emb.device, 
#                                    dtype=word_emb.dtype)
#             if self.clamp_len > 0:
#                 pos_seq.clamp_(max=self.clamp_len)
#             pos_emb = self.pos_emb(pos_seq)

#             core_out = self.drop(word_emb)
#             pos_emb = self.drop(pos_emb)

#             hids.append(core_out)
#             print('core out isna ', core_out.isnan().sum())
#             for i, layer in enumerate(self.layers):
#                 mems_i = None if mems is None else mems[i]
#                 core_out = layer(core_out, pos_emb, self.r_w_bias,
#                         self.r_r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
#                 hids.append(core_out)
#                 print('core out isna ', core_out.isnan().sum())
#         elif self.attn_type == 1: # learnable
#             core_out = self.drop(word_emb)
#             hids.append(core_out)
#             for i, layer in enumerate(self.layers):
#                 if self.clamp_len > 0:
#                     r_emb = self.r_emb[i][-self.clamp_len :]
#                     r_bias = self.r_bias[i][-self.clamp_len :]
#                 else:
#                     r_emb, r_bias = self.r_emb[i], self.r_bias[i]

#                 mems_i = None if mems is None else mems[i]
#                 core_out = layer(core_out, r_emb, self.r_w_bias[i],
#                         r_bias, dec_attn_mask=dec_attn_mask, mems=mems_i)
#                 hids.append(core_out)
#         elif self.attn_type == 2: # absolute
#             pos_seq = torch.arange(klen - 1, -1, -1.0, device=word_emb.device,
#                                    dtype=word_emb.dtype)
#             if self.clamp_len > 0:
#                 pos_seq.clamp_(max=self.clamp_len)
#             pos_emb = self.pos_emb(pos_seq)

#             core_out = self.drop(word_emb + pos_emb[-qlen:])

#             hids.append(core_out)
#             for i, layer in enumerate(self.layers):
#                 mems_i = None if mems is None else mems[i]
#                 if mems_i is not None and i == 0:
#                     mems_i += pos_emb[:mlen]
#                 core_out = layer(core_out, dec_attn_mask=dec_attn_mask,
#                                  mems=mems_i)
#                 hids.append(core_out)
#         elif self.attn_type == 3:
#             core_out = self.drop(word_emb)

#             hids.append(core_out)
#             for i, layer in enumerate(self.layers):
#                 mems_i = None if mems is None else mems[i]
#                 if mems_i is not None and mlen > 0:
#                     cur_emb = self.r_emb[i][:-qlen]
#                     cur_size = cur_emb.size(0)
#                     if cur_size < mlen:
#                         cur_emb_pad = cur_emb[0:1].expand(mlen-cur_size, -1, -1)
#                         cur_emb = torch.cat([cur_emb_pad, cur_emb], 0)
#                     else:
#                         cur_emb = cur_emb[-mlen:]
#                     mems_i += cur_emb.view(mlen, 1, -1)
#                 core_out += self.r_emb[i][-qlen:].view(qlen, 1, -1)

#                 core_out = layer(core_out, dec_attn_mask=dec_attn_mask,
#                                  mems=mems_i)
#                 hids.append(core_out)

#         core_out = self.drop(core_out)

#         new_mems = self._update_mems(hids, mems, mlen, qlen)

#         return core_out, new_mems

#     def forward(self, data, target, *mems):
#         # nn.DataParallel does not allow size(0) tensors to be broadcasted.
#         # So, have to initialize size(0) mems inside the model forward.
#         # Moreover, have to return new_mems to allow nn.DataParallel to piece
#         # them together.
#         if not mems: mems = self.init_mems()

#         # if self.num_mem_tokens not in (0, None):
#         #     mem_tokens = self.mem_tokens.expand(1, *self.mem_tokens.shape).clone()
#         #     memory = torch.cat([mem_tokens] * data.shape[0], dim=0)
#         #     data = torch.cat((memory, data), dim=1)

#         tgt_len = target.size(0)
#         hidden, new_mems = self._forward(data, mems=mems)

#         print('hidden isnan ', torch.isnan(hidden).sum())
#         print('mems ', torch.isnan(mems[0]).sum())
#         # print('hidden, new mems: ', (len(hidden), len(new_mems)))
#         pred_hid = hidden[-tgt_len:]
#         if self.sample_softmax > 0 and self.training:
#             assert self.tie_weight
#             logit = sample_logits(self.word_emb,
#                 self.out_layer.bias, target, pred_hid, self.sampler)
#             loss = -F.log_softmax(logit, -1)[:, :, 0]
#         else:
#             # print('pred_hid ', pred_hid.shape)
#             if self.num_mem_tokens not in (0, None):
#                 mem_tokens, pred_hid = pred_hid[:, :self.num_mem_tokens].clone(), pred_hid[:, self.num_mem_tokens:].clone()
#             # print('pred_hid ', pred_hid.shape)
#             # print('target',  target.shape)
#             # print('view pred', pred_hid.view(-1, pred_hid.size(-1)).shape)
#             # print('view target ', target.view(-1).shape)
#             loss = self.crit(pred_hid.view(-1, pred_hid.size(-1)), target.view(-1))
#             # print('loss ', loss.shape)
#             loss = loss.view(tgt_len, -1)
#             # print('loss ', loss.shape)

#            # pred_hid = torch.cat((mem_tokens, pred_hid), dim=1)

#         output = [loss]

#         if new_mems is not None:
#             output += new_mems
#         if self.num_mem_tokens not in (0, None):
#             output = [mem_tokens] + output
            
#         return output


In [7]:
num_mem_tokens = 16

div_val = 2
d_embed = 100

cutoffs = [args.n_token // 2]
tie_projs = [False] + [True] * len(cutoffs)

model = MemTransformerLM(args.n_token, args.n_layer, args.n_head,
                        args.d_model, args.d_head, args.d_inner, args.dropout,
                        dropatt=args.dropout, tie_weight=True, 
                        d_embed=d_embed, div_val=div_val, 
                        tie_projs=tie_projs, pre_lnorm=True,
                        tgt_len=tgt_len, ext_len=ext_len, mem_len=mem_len,# num_mem_tokens=num_mem_tokens,
                        cutoffs=cutoffs, attn_type=0).to(device)


# model_mem = MemTransformerLM_mem(args.n_token, args.n_layer, args.n_head,
                        # args.d_model, args.d_head, args.d_inner, args.dropout,
                        # dropatt=args.dropout, tie_weight=True, 
                        # d_embed=d_embed, div_val=div_val, 
                        # tie_projs=tie_projs, pre_lnorm=True,
                        # tgt_len=tgt_len, ext_len=ext_len, mem_len=mem_len, num_mem_tokens=num_mem_tokens,
                        # cutoffs=cutoffs, attn_type=0).to(device)

In [8]:
# model.layers

In [9]:
# for p in model.parameters():
#     print(p.shape)

In [10]:
mems = tuple()
idx, (inp, tgt, seqlen) = list(enumerate(diter))[0]
out = model(inp, tgt, *mems, )
print(len(out))
mem_tokens, enc, mems = out[0], out[1], out[1:]

# print(out[0].shape, [m.shape for m in out[1:]], len(out[1:]))

# # print(mems[-1])

# # idx, (inp, tgt, seqlen) = list(enumerate(diter))[0]
# # out = model(inp, tgt, *mems, )
# # enc, mems = out[0], out[1:]
# # print(out[0].shape, [m.shape for m in out[1:]], len(out[1:]))

4


/home/ayd98/Desktop/MIPT/TransformerXL/transformer-xl/pytorch/mem_transformer.py:269: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1104.)
  attn_score = attn_score.float().masked_fill(


In [43]:
mems = tuple()
idx, (inp, tgt, seqlen) = list(enumerate(diter))[0]
out = model(inp, tgt, *mems, )
print(len(out))
enc = out[0]

4


In [44]:
len(out_mem)

5

In [51]:
i = 3
out_mem[i+1] - out[i]

RuntimeError: The size of tensor a (20) must match the size of tensor b (4) at non-singleton dimension 1

In [12]:
enc

tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [n

In [21]:
enc

tensor([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [n

In [252]:
torch.isnan(enc).sum()

tensor(144)

In [248]:
inp

tensor([[8456, 6490, 7959, 8298],
        [ 778, 9517, 3080, 2191],
        [8833, 6181, 2880, 2668],
        [5183, 6818, 7858, 6641],
        [6983, 5891, 3529, 2450],
        [1939, 7080, 2023, 7603],
        [2674, 7028, 3830, 2783],
        [7772, 8202, 2321, 7247],
        [3748, 5586, 4165, 6088],
        [4639, 2829, 1936, 7271],
        [8479, 4443, 9749, 6767],
        [5289, 9458, 8027, 8781],
        [8850, 1741, 1721, 4034],
        [4498, 9962, 9225, 1992],
        [8376, 9134, 3195, 7744],
        [2113, 1467,  754, 8523],
        [4358, 9983, 1189, 3312],
        [9569, 7361, 5904,  688],
        [9976, 7937, 1024, 9200],
        [1928, 4606, 2899, 1081],
        [4204, 3628, 7964, 4120],
        [3171, 8327, 8265, 3809],
        [4507, 4815, 7846, 4235],
        [4829, 1950, 6476, 9920],
        [7774, 7801, 4840, 4767],
        [9865, 4135, 3519, 3951],
        [7903,  403, 3695, 9756],
        [1005, 1072, 7007, 1459],
        [3238, 6822, 1936, 8168],
        [3985,

In [247]:
mems

[tensor([[nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
         [nan, nan, nan, nan],
        

In [78]:
print(inp.shape, tgt.shape, seqlen)
inp[0], tgt[0]

torch.Size([36, 4]) torch.Size([36, 4]) 36


(tensor([8456, 6490, 7959, 8298]), tensor([ 778, 9517, 3080, 2191]))

In [79]:
out[0].shape, [m.shape for m in out[1:]], len(out[1:])

(torch.Size([36, 4]),
 [torch.Size([36, 4, 202]),
  torch.Size([36, 4, 202]),
  torch.Size([36, 4, 202])],
 3)

In [84]:
out[-1]

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        ...,

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, 

In [10]:
# print(sum(p.numel() for p in model.parameters()))
# print([p.numel() for p in model.parameters()])
# data.shape

In [11]:
# data[:38]

In [12]:
# list(enumerate(diter))[0][1][0]

In [13]:
# for i, v in enumerate(diter):
#     print(i, v[0].shape, v[1].shape, v[2])

In [14]:

data = torch.LongTensor(data_len*B).random_(0, args.n_token).to(device)
diter = data_utils.LMOrderedIterator(data, B, tgt_len, device=device, ext_len=ext_len)

cutoffs = [args.n_token // 2]
tie_projs = [False] + [True] * len(cutoffs)

for div_val in [1, 2]:
    for d_embed in [200, 100]:
        model = MemTransformerLM(args.n_token, args.n_layer, args.n_head,
                        args.d_model, args.d_head, args.d_inner, args.dropout,
                        dropatt=args.dropout, tie_weight=True, 
                        d_embed=d_embed, div_val=div_val, 
                        tie_projs=tie_projs, pre_lnorm=True,
                        tgt_len=tgt_len, ext_len=ext_len, mem_len=mem_len, 
                        cutoffs=cutoffs, attn_type=0).to(device)

        print(sum(p.numel() for p in model.parameters()))

        mems = tuple()
        for idx, (inp, tgt, seqlen) in enumerate(diter):
            print('batch {}'.format(idx))
            out = model(inp, tgt, *mems)
            mems = out[1:]


2351009
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
1390909
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
1951009
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
1150909
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
